# Intro to Spark and PySpark


In [8]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pathlib import Path

## Reading parquets

Instantiate a spark session

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

23/02/17 22:42:34 WARN Utils: Your hostname, Kohada resolves to a loopback address: 127.0.1.1; using 172.30.125.167 instead (on interface eth0)
23/02/17 22:42:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/17 22:42:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wc -l ../data/taxi_ingest_data/fhvhv_tripdata_2021-01.parquet

1006794 ../data/taxi_ingest_data/fhvhv_tripdata_2021-01.parquet


Spark session reads the parquet in as `pyspark.sql.Dataframe`

In [6]:
fpath = Path("../data/taxi_ingest_data/fhvhv_tripdata_2021-01.parquet")
df = spark.read \
    .option('header', 'true') \
    .parquet(str(fpath))
df.show(10)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

The sparks UI can be accessed via http://localhost:4040/jobs/

![spark-ui](./img/spark-ui-1.png)

In [ ]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(originating_base_num,StringType,true),StructField(request_datetime,TimestampType,true),StructField(on_scene_datetime,TimestampType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(trip_miles,DoubleType,true),StructField(trip_time,LongType,true),StructField(base_passenger_fare,DoubleType,true),StructField(tolls,DoubleType,true),StructField(bcf,DoubleType,true),StructField(sales_tax,DoubleType,true),StructField(congestion_surcharge,DoubleType,true),StructField(airport_fee,DoubleType,true),StructField(tips,DoubleType,true),StructField(driver_pay,DoubleType,true),StructField(shared_request_flag,StringType,true),StructField(shared_match_flag,StringType,true),StructField(access_a_ride_flag,StringType,true),StructField(wav_req

Formatted:

```scala
StructType(List(
    StructField(hvfhs_license_num,StringType,true),
    StructField(dispatching_base_num,StringType,true),
    StructField(originating_base_num,StringType,true),
    StructField(request_datetime,TimestampType,true),
    StructField(on_scene_datetime,TimestampType,true),
    StructField(pickup_datetime,TimestampType,true),
    StructField(dropoff_datetime,TimestampType,true),
    StructField(PULocationID,LongType,true),
    StructField(DOLocationID,LongType,true),
    StructField(trip_miles,DoubleType,true),
    StructField(trip_time,LongType,true),
    StructField(base_passenger_fare,DoubleType,true),
    StructField(tolls,DoubleType,true),
    StructField(bcf,DoubleType,true),
    StructField(sales_tax,DoubleType,true),
    StructField(congestion_surcharge,DoubleType,true),
    StructField(airport_fee,DoubleType,true),
    StructField(tips,DoubleType,true),
    StructField(driver_pay,DoubleType,true),
    StructField(shared_request_flag,StringType,true),
    StructField(shared_match_flag,StringType,true),
    StructField(access_a_ride_flag,StringType,true),
    StructField(wav_request_flag,StringType,true),
    StructField(wav_match_flag,StringType,true)
    )
)
```

- `scala` syntax
- Double: float (8 bytes)
- Long: int64 (8 bytes)
- `true` refers to `is_nullable`

So contrary to the `.csv`, the parquet types seem to be correct.

Otherwise, Spark can read directly from a `pd.DataFrame`, where we can enforce the schema before feeding to spark

In [14]:
!head -n 1001 ../data/taxi_ingest_data/fhvhv_tripdata_2021-01.parquet > ../data/taxi_ingest_data/fhvhv_tripdata_sample.parquet
!tail -n 1 ../data/taxi_ingest_data/fhvhv_tripdata_2021-01.parquet >> ../data/taxi_ingest_data/fhvhv_tripdata_sample.parquet


In [16]:
try:
    df_pd = pd.read_parquet(fpath.parent / 'fhv_tripdata_sample.parquet',)
    df_pd.dtypes
except OSError as e:
    print("Difficult to read only parts of a parquet with only pandas\n", e)

Difficult to read only parts of a parquet with only pandas
 Invalid column metadata (corrupt file?)


Use `pyarrow` directly to access the `nrows` arg

[SOF source](https://stackoverflow.com/a/69888274/5496416)

In [18]:
from pyarrow.parquet import ParquetFile
import pyarrow as pa

# reader interface
pf = ParquetFile(fpath)
first_nrows = next(pf.iter_batches(batch_size=1000))
df_pd = pa.Table.from_batches([first_nrows]).to_pandas()
df_pd.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [22]:
df_from_pd = spark.createDataFrame(df_pd)
df_from_pd.head(3)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=nan, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOL

We can extract the `scala` types above and convert it to python via `pyspark.sql.types`, and revise the schema during read. E.g. change `locationID` to `Integer` instead of `Long` to reduce storage requirements, since `Long` takes up 8 bytes (int64) and `Integer` takes up 4 (int32)

In [46]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('originating_base_num', types.StringType(), True),
    types.StructField('request_datetime', types.TimestampType(), True),
    types.StructField('on_scene_datetime', types.TimestampType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('trip_miles', types.DoubleType(), True),
    types.StructField('trip_time', types.LongType(), True),
    types.StructField('base_passenger_fare', types.DoubleType(), True),
    types.StructField('tolls', types.DoubleType(), True),
    types.StructField('bcf', types.DoubleType(), True),
    types.StructField('sales_tax', types.DoubleType(), True),
    types.StructField('congestion_surcharge', types.DoubleType(), True),
    types.StructField('airport_fee', types.DoubleType(), True),
    types.StructField('tips', types.DoubleType(), True),
    types.StructField('driver_pay', types.DoubleType(), True),
    types.StructField('shared_request_flag', types.StringType(), True),
    types.StructField('shared_match_flag', types.StringType(), True),
    types.StructField('access_a_ride_flag', types.StringType(), True),
    types.StructField('wav_request_flag', types.StringType(), True),
    types.StructField('wav_match_flag', types.StringType(), True)
    ]
)


In [47]:
df_with_schema = spark.read \
    .option('header', 'true') \
    .schema(schema=schema) \
    .parquet(str(fpath))

In [48]:
df_with_schema.head(3)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 8, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 8, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 8, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 8, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 8, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 8, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 8, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 9, 18, 21), PULocationID=152, DO

Couldn't convert from `LongType` to `IntegerType`; parquet complained about expecting `INT64`. Stay as `Long`.

## Repartition

The whole idea of spark is distributing the dataset and parallelizing the task between the executors in the cluster. Thus we should break out our parquet into smaller *partitions*  so that the workload can be more easily distributed.

Otherwise, one executor in the cluster will receive the one dataset, work on it by itself, while the other executors sit idle.

Use `df.repartition(int)`

In [49]:
# lazy eval; will not execute yet
# returns a new df that is hash partitioned
df_parts = df_with_schema.repartition(24)

Do not create the directory for `df.write.parquet()`; it will raise `Path already exists` error. Let it create it on its own

In [51]:
partition_dir = fpath.parent / 'fhvhv' / 'parts'
# if not partition_dir.exists():
#     partition_dir.mkdir(parents=True)
# write our df into 24 partitions
df_parts.write.parquet(str(partition_dir))

Output:

```bash
kohada@Kohada:~/de-zoomcamp/data/taxi_ingest_data$ ls fhvhv/parts/ -lh
total 526M
-rw-r--r-- 1 kohada kohada   0 Feb 18 00:40 _SUCCESS
-rw-r--r-- 1 kohada kohada 22M Feb 18 00:39 part-00000-0c7da773-7f90-44bc-9962-54679a206388-c000.snappy.parquet
-rw-r--r-- 1 kohada kohada 22M Feb 18 00:39 part-00001-0c7da773-7f90-44bc-9962-54679a206388-c000.snappy.parquet
-rw-r--r-- 1 kohada kohada 22M Feb 18 00:39 part-00002-0c7da773-7f90-44bc-9962-54679a206388-c000.snappy.parquet
-rw-r--r-- 1 kohada kohada 22M Feb 18 00:39 part-00003-0c7da773-7f90-44bc-9962-54679a206388-c000.snappy.parquet
...
-rw-r--r-- 1 kohada kohada 22M Feb 18 00:40 part-00023-0c7da773-7f90-44bc-9962-54679a206388-c000.snappy.parquet
```